In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cellpose import models
from cellpose.io import imread
import glob
from pathlib import Path
from PIL import Image, ImageSequence
from tqdm import tqdm
import os
import os.path


In [ ]:

# model_type='cyto' or 'nuclei' or 'cyto2'
# model = models.Cellpose(gpu=True, model_type="cyto")
model = models.CellposeModel(pretrained_model='src/cxa_segmentation/notebooks/models/yuzhong_cyto2', gpu=True)
# list of files
# PUT PATH TO YOUR FILES HERE!
dir_path = Path('D:\\xing-vimentin-dic-pipeline\\src\\cxa_segmentation\\cxa-data\\june_2022_data\\day0_Notreat_Group1_wellA1_RI_MIP_stitched')
img_list = sorted(glob.glob(str(dir_path/"*tif")))

In [ ]:
img_list[:4]

In [ ]:
def segment_single_image_by_cellpose(image, model, channels = [[0, 0]], diameter=150):
    result_tuple = model.eval([image], diameter=diameter, channels=channels)
    masks = result_tuple[0]
    return masks[0]

def segment_single_images_by_cellpose(images, model, channels = [[0, 0]], diameter=150):
    masks, flows, styles, diams = model.eval(images, diameter=diameter, channels=channels)
    return masks


In [ ]:
def gen_T_filename(frame, ndigits):
    return "T%s.tif" % (str(frame).zfill(ndigits))
ndigits = len(str(len(img_list)))
print("ndigits:", ndigits)
out_dir = Path("./yuzhong_cellpose_cyto2_segs")

for img_path in tqdm(img_list):
    file_name = os.path.basename(img_path)
    output_filename = file_name.split(".")[0] + ".png" # change extension to PNG
    img = imread(img_path) # use imread from cellpose
    mask = segment_single_image_by_cellpose(img, model, diameter=150)
    temp_img = Image.fromarray(mask)
    temp_img.save(out_dir / output_filename)